In [1]:
import pandas as pd
import numpy as np
import recordlinkage
from recordlinkage.preprocessing import clean
from itertools import product
from rapidfuzz import process, fuzz

In [2]:
df = pd.read_csv("../Data/Original_Data/Olist_Brazillian_Ecommerce_Dataset.csv")
print(df.head())

   Unnamed: 0                          order_id  order_item_id  \
0           0  00010242fe8c5a6d1ba2dd792cb16214              1   
1           1  130898c0987d1801452a8ed92a670612              1   
2           2  532ed5e14e24ae1f0d735b91524b98b9              1   
3           3  6f8c31653edb8c83e1a739408b5ff750              1   
4           4  7d19f4ef4d04461989632411b7e588b9              1   

                        customer_id                customer_unique_id  \
0  3ce436f183e68e07877b285a838db11a  871766c5855e863f6eccc05f988b23cb   
1  e6eecc5a77de221464d1c4eaff0a9b64  0fb8e3eab2d3e79d92bb3fffbb97f188   
2  4ef55bf80f711b372afebcb7c715344a  3419052c8c6b45daf79c1e426f9e9bcb   
3  30407a72ad8b3f4df4d15369126b20c9  e7c828d22c0682c1565252deefbe334d   
4  91a792fef70ecd8cc69d3c7feb3d12da  0bb98ba72dcc08e95f9d8cc434e9a2cc   

   customer_zip_code_prefix          customer_city customer_state  \
0                     28013  campos dos goytacazes             RJ   
1                     7580

In [3]:
df = df.drop(columns={'Unnamed: 0', 'order_unique_id', 'day_of_purchase', 'month_of_purchase', 'year_of_purchase', 'month/year_of_purchase'})

In [4]:
df.shape

(113390, 33)

## Standardizing column names

In [5]:
df.columns = df.columns.str.strip()
df.columns = df.columns.str.lower().str.replace(' ', '_')

In [6]:
df.columns

Index(['order_id', 'order_item_id', 'customer_id', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_id', 'seller_city', 'seller_state', 'seller_zip_code_prefix',
       'payment_type', 'payment_sequential', 'payment_installments', 'price',
       'freight_value', 'payment_value', 'shipping_limit_date',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_status'],
      dtype='object')

In [7]:
df = df.rename(columns = {'product_name_lenght': 'product_name_length',
                          'product_description_lenght': 'product_description_length'
                         })

# 1. Assessing Data Quality

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113390 entries, 0 to 113389
Data columns (total 33 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       113390 non-null  object 
 1   order_item_id                  113390 non-null  int64  
 2   customer_id                    113390 non-null  object 
 3   customer_unique_id             113390 non-null  object 
 4   customer_zip_code_prefix       113390 non-null  int64  
 5   customer_city                  113390 non-null  object 
 6   customer_state                 113390 non-null  object 
 7   product_id                     113390 non-null  object 
 8   product_category_name          113390 non-null  object 
 9   product_name_length            113390 non-null  float64
 10  product_description_length     113390 non-null  float64
 11  product_photos_qty             113390 non-null  float64
 12  product_weight_g              

In [9]:
original_df_length = len(df)
print(original_df_length)

113390


### Identifying duplicate rows in the dataset

In [10]:
duplicate_rows = df[df.duplicated()]
if duplicate_rows.empty:
    print("There are no duplicate rows")
else:
    print(duplicate_rows)

There are no duplicate rows


### Identifying Constant/quasi-constant columns

In [11]:
constant_cols = []
for col in df.columns:
    if df[col].nunique == 1:
        constant_cols.append(col)

# print(constant_cols)

if constant_cols:
    print(f"Columns with all unique values (Constant Columns): {constant_cols}")
else:
    print("There are no constant columns")

There are no constant columns


### Identifying inconsistent data format

In [12]:
# Checking dtype of any date columns
date_cols = []
for col in df.columns:
    if 'date' in col:
        date_cols.append(col)

print(f"Date columns in the dataset: {date_cols}")

Date columns in the dataset: ['shipping_limit_date', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']


In [13]:
for col in date_cols:
    print(f"{col} datatype: {df[col].dtype}")

shipping_limit_date datatype: object
order_delivered_carrier_date datatype: object
order_delivered_customer_date datatype: object
order_estimated_delivery_date datatype: object


In [14]:
# Checking dtype of any boolean columns
bool_cols = []
for col in df.columns:
    if set(df[col].unique()) == {0, 1}:
        bool_cols.append(col)

if bool_cols:
    print(f"Boolean columns in dataset: {bool_cols}")
else:
    print("There are no boolean columns in the dataset")

There are no boolean columns in the dataset


### Identifying any errors/anomalies in int columns

In [15]:
int_cols = df.select_dtypes(include={'int64'})
print(int_cols)

        order_item_id  customer_zip_code_prefix  seller_zip_code_prefix  \
0                   1                     28013                   27277   
1                   1                     75800                   27277   
2                   1                     30720                   27277   
3                   1                     83070                   27277   
4                   1                     36400                   27277   
...               ...                       ...                     ...   
113385              1                     22793                   80220   
113386              1                     13485                   82620   
113387              1                     29066                   86026   
113388              1                     12280                    3124   
113389              2                     12280                    3124   

        payment_sequential  payment_installments  
0                        1                     2

In [16]:
df['order_item_id'].value_counts()

order_item_id
1     99299
2      9930
3      2301
4       948
5       445
6       253
7        60
8        35
9        28
10       25
11       17
12       13
13        8
14        7
15        5
16        3
17        3
18        3
19        3
20        3
21        1
Name: count, dtype: int64

In [17]:
df['payment_sequential'].value_counts()

payment_sequential
1     108555
2       3252
3        625
4        300
5        178
6        122
7         83
8         53
9         43
10        37
11        33
12        25
13        14
14        12
15        10
17         8
16         8
18         8
19         8
21         5
20         5
22         2
26         1
25         1
24         1
23         1
Name: count, dtype: int64

In [18]:
df['payment_installments'].value_counts()

payment_installments
1     56497
2     13199
3     11331
4      7713
10     6632
5      5827
8      4869
6      4456
7      1743
9       701
12      160
15       89
18       38
24       34
11       25
20       20
13       17
14       15
16        7
17        7
21        5
0         3
23        1
22        1
Name: count, dtype: int64

### Identifying any errors/anomalies in float columns

In [19]:
float_cols = df.select_dtypes(include={'float64'})
print(float_cols)

        product_name_length  product_description_length  product_photos_qty  \
0                      58.0                       598.0                 4.0   
1                      58.0                       598.0                 4.0   
2                      58.0                       598.0                 4.0   
3                      58.0                       598.0                 4.0   
4                      58.0                       598.0                 4.0   
...                     ...                         ...                 ...   
113385                 44.0                       533.0                 2.0   
113386                 32.0                      1306.0                 3.0   
113387                 50.0                      1935.0                 4.0   
113388                 28.0                       242.0                 1.0   
113389                 28.0                       242.0                 1.0   

        product_weight_g  product_length_cm  produc

### Identifying any errors/anomalies in Cateogrical/Object columns

In [20]:
object_cols = df.select_dtypes(include={'object'})
print(object_cols)

                                order_id                       customer_id  \
0       00010242fe8c5a6d1ba2dd792cb16214  3ce436f183e68e07877b285a838db11a   
1       130898c0987d1801452a8ed92a670612  e6eecc5a77de221464d1c4eaff0a9b64   
2       532ed5e14e24ae1f0d735b91524b98b9  4ef55bf80f711b372afebcb7c715344a   
3       6f8c31653edb8c83e1a739408b5ff750  30407a72ad8b3f4df4d15369126b20c9   
4       7d19f4ef4d04461989632411b7e588b9  91a792fef70ecd8cc69d3c7feb3d12da   
...                                  ...                               ...   
113385  fdaf98feac227eb978d8f33f36e0231d  81ae83ce1a10af9f93fa3512bd9713b1   
113386  fe68b52db13993f58175fa589125d345  94203a446c1b46b5ac95b0aaf7221804   
113387  fefacc66af859508bf1a7934eab1e97f  f48d464a0baaea338cb25f816991ab1f   
113388  ff701a7c869ad21de22a6994237c8a00  8f6d16c31abb30b7b4650272cb0014a3   
113389  ff701a7c869ad21de22a6994237c8a00  8f6d16c31abb30b7b4650272cb0014a3   

                      customer_unique_id          customer_city

In [21]:
print(object_cols.columns)

Index(['order_id', 'customer_id', 'customer_unique_id', 'customer_city',
       'customer_state', 'product_id', 'product_category_name', 'seller_id',
       'seller_city', 'seller_state', 'payment_type', 'shipping_limit_date',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'order_status'],
      dtype='object')


In [22]:
print(df['payment_type'].value_counts())

payment_type
credit_card    83706
boleto         22047
voucher         6012
debit_card      1625
Name: count, dtype: int64


In [23]:
print(df['shipping_limit_date'].value_counts())

shipping_limit_date
2017-08-14 20:43:31    63
2017-10-05 17:44:41    38
2017-06-15 16:15:08    26
2018-02-27 12:28:15    24
2018-05-15 15:30:28    24
                       ..
2018-07-12 20:10:51     1
2018-06-19 22:00:37     1
2018-01-18 07:26:34     1
2018-04-05 12:15:28     1
2017-08-16 12:30:15     1
Name: count, Length: 90127, dtype: int64


# 2. Data Cleaning

## Numeric Features -> Identifying discrepancies/inconsistencies 

In [24]:
print(f"Number of int columns: {len(int_cols.columns)}")
for col in int_cols:
    print(f"{col} Min value: {df[col].min()} | Max value: {df[col].max()} | Null values: {df[col].isna().sum()}")

Number of int columns: 5
order_item_id Min value: 1 | Max value: 21 | Null values: 0
customer_zip_code_prefix Min value: 1003 | Max value: 99980 | Null values: 0
seller_zip_code_prefix Min value: 1001 | Max value: 99730 | Null values: 0
payment_sequential Min value: 1 | Max value: 26 | Null values: 0
payment_installments Min value: 0 | Max value: 24 | Null values: 0


- All integer columns contain no null/missing values
- All integer columns' values fall within reasonable/logical range

In [25]:
print(f"Number of float columns: {len(float_cols.columns)}")
for col in float_cols:
    print(f"{col} - Min value: {df[col].min()} | Max value: {df[col].max()} | Null values: {df[col].isna().sum()}")

Number of float columns: 10
product_name_length - Min value: 5.0 | Max value: 76.0 | Null values: 0
product_description_length - Min value: 4.0 | Max value: 3992.0 | Null values: 0
product_photos_qty - Min value: 1.0 | Max value: 20.0 | Null values: 0
product_weight_g - Min value: 0.0 | Max value: 40425.0 | Null values: 0
product_length_cm - Min value: 7.0 | Max value: 105.0 | Null values: 0
product_height_cm - Min value: 2.0 | Max value: 105.0 | Null values: 0
product_width_cm - Min value: 6.0 | Max value: 118.0 | Null values: 0
price - Min value: 0.85 | Max value: 6735.0 | Null values: 0
freight_value - Min value: 0.0 | Max value: 409.68 | Null values: 0
payment_value - Min value: 0.0 | Max value: 13664.08 | Null values: 0


- All float columns contain no null/missing values
- All float columns' values fall within reasonable/logical range

## Converting date objects to datetime dtype

In [26]:
date_cols = []
for col in df.columns:
    if 'date' in col:
        date_cols.append(col)

if date_cols:
    print(f"There are {len(date_cols)} Date columns in the dataset:\n{date_cols}")
else:
    print("There are no date columns in the dataset")

There are 4 Date columns in the dataset:
['shipping_limit_date', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']


In [27]:
date_cols.append('order_purchase_timestamp')
date_cols.append('order_approved_at')

In [28]:
print(f"There are {len(date_cols)} Date columns in the dataset")
for col in date_cols:
    df[col] = pd.to_datetime(df[col])
    print(f"{col} converted to datatype: {df[col].dtype}")

There are 6 Date columns in the dataset
shipping_limit_date converted to datatype: datetime64[ns]
order_delivered_carrier_date converted to datatype: datetime64[ns]
order_delivered_customer_date converted to datatype: datetime64[ns]
order_estimated_delivery_date converted to datatype: datetime64[ns]
order_purchase_timestamp converted to datatype: datetime64[ns]
order_approved_at converted to datatype: datetime64[ns]


## Categorical Features -> Identifying discrepancies/inconsistencies

In [29]:
id_cols = []
for col in object_cols:
    if col.endswith("_id"):
        id_cols.append(col)

if id_cols:
    print(f"There are {len(id_cols)} ID columns in the dataset:\n{id_cols}")
else:
    print("There are no id columns in the dataset")

There are 5 ID columns in the dataset:
['order_id', 'customer_id', 'customer_unique_id', 'product_id', 'seller_id']


In [30]:
for col in id_cols:
    print(f"{col} has:\n{df[col].nunique()} unique values and {df[col].isna().sum()} null values\n")

order_id has:
95128 unique values and 0 null values

customer_id has:
95128 unique values and 0 null values

customer_unique_id has:
92081 unique values and 0 null values

product_id has:
31625 unique values and 0 null values

seller_id has:
2914 unique values and 0 null values



- `order_id`: There are 95128 unique orders in the dataset
- `customer_id`: There are 95128 customers that purchased an order in the dataset
- `customer_unique_id`: There are 92081 unique customers in the dataset
- `product_id`: There are 31625 unique products in the dataset
- `seller_id`: There are 2914 unique sellers in the dataset

In [31]:
cat_cols = set(object_cols) - set(date_cols)
cat_cols = set(cat_cols) - set(id_cols)
print(f"There are {len(cat_cols)} categorical columns in the dataset:\n{cat_cols}")

# customer state, sellet state, order status, payment type, product category name, customer city, seller city.

There are 7 categorical columns in the dataset:
{'seller_state', 'customer_state', 'product_category_name', 'order_status', 'seller_city', 'customer_city', 'payment_type'}


### Customer State

In [32]:
print(df['customer_state'].value_counts())
print(f"Total sum of vlaues in 'customer_state' is: {df['customer_state'].value_counts().sum()}")

customer_state
SP    47830
RJ    14651
MG    13234
RS     6284
PR     5790
SC     4161
BA     3859
DF     2389
GO     2319
ES     2288
PE     1804
CE     1482
PA     1071
MT     1067
MS      826
MA      808
PB      614
RN      555
PI      546
AL      437
SE      384
TO      330
RO      275
AM      167
AC       92
AP       83
RR       44
Name: count, dtype: int64
Total sum of vlaues in 'customer_state' is: 113390


### Seller State

In [33]:
print(df['seller_state'].value_counts())
print(f"Total sum of vlaues in 'seller_state' is: {df['seller_state'].value_counts().sum()}")

seller_state
SP    80870
MG     8815
PR     8733
RJ     4798
SC     4160
RS     2203
DF      919
BA      679
GO      527
PE      462
MA      406
ES      371
MT      146
CE       99
MS       60
RN       56
PB       40
RO       14
PI       11
SE       10
PA        8
AM        3
Name: count, dtype: int64
Total sum of vlaues in 'seller_state' is: 113390


### Order Status

In [34]:
print(df['order_status'].value_counts())
print(f"Total sum of vlaues in 'order_status' is: {df['order_status'].value_counts().sum()}")

order_status
delivered    113383
canceled          7
Name: count, dtype: int64
Total sum of vlaues in 'order_status' is: 113390


### Payment Type

In [35]:
print(df['payment_type'].value_counts())
print(f"Total sum of vlaues in 'payment_type' is: {df['payment_type'].value_counts().sum()}")

payment_type
credit_card    83706
boleto         22047
voucher         6012
debit_card      1625
Name: count, dtype: int64
Total sum of vlaues in 'payment_type' is: 113390


### Product Category Name

In [36]:
pd.set_option('display.max_rows', None)
print(df['product_category_name'].value_counts())
print(f"Total sum of vlaues in 'product_category_name' is: {df['product_category_name'].value_counts().sum()}")

product_category_name
cama_mesa_banho                                   11649
beleza_saude                                       9761
esporte_lazer                                      8731
moveis_decoracao                                   8553
informatica_acessorios                             7897
utilidades_domesticas                              7172
relogios_presentes                                 6063
telefonia                                          4601
ferramentas_jardim                                 4463
automotivo                                         4283
brinquedos                                         4185
cool_stuff                                         3904
perfumaria                                         3480
bebes                                              3111
eletronicos                                        2807
papelaria                                          2564
fashion_bolsas_e_acessorios                        2106
pet_shop                  

#### Translating product category names to english version

In [37]:
product_cat_names = df['product_category_name'].unique()

In [38]:
print(len(product_cat_names), product_cat_names)

73 ['cool_stuff' 'brinquedos' 'tablets_impressao_imagem' 'pet_shop'
 'moveis_decoracao' 'cama_mesa_banho' 'perfumaria'
 'informatica_acessorios' 'utilidades_domesticas' 'papelaria'
 'ferramentas_jardim' 'esporte_lazer' 'telefonia' 'audio' 'beleza_saude'
 'construcao_ferramentas_jardim' 'relogios_presentes' 'bebes'
 'livros_tecnicos' 'dvds_blu_ray' 'consoles_games' 'alimentos'
 'fashion_bolsas_e_acessorios' 'casa_conforto' 'moveis_sala'
 'casa_conforto_2' 'instrumentos_musicais' 'automotivo'
 'moveis_escritorio' 'casa_construcao' 'construcao_ferramentas_iluminacao'
 'eletronicos' 'sinalizacao_e_seguranca' 'cine_foto'
 'construcao_ferramentas_construcao' 'telefonia_fixa' 'fraldas_higiene'
 'pc_gamer' 'fashion_roupa_masculina' 'eletroportateis'
 'industria_comercio_e_negocios' 'construcao_ferramentas_seguranca'
 'fashion_esporte' 'malas_acessorios' 'market_place'
 'construcao_ferramentas_ferramentas' 'eletrodomesticos'
 'agro_industria_e_comercio' 'climatizacao' 'artes' 'la_cuisine'
 'por

In [39]:
translation = pd.read_csv("../Data/Original_Data/product_category_name_translation.csv")
translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [40]:
product_cat_names_trans = translation['product_category_name'].unique()

In [41]:
print(len(product_cat_names_trans), product_cat_names_trans)

71 ['beleza_saude' 'informatica_acessorios' 'automotivo' 'cama_mesa_banho'
 'moveis_decoracao' 'esporte_lazer' 'perfumaria' 'utilidades_domesticas'
 'telefonia' 'relogios_presentes' 'alimentos_bebidas' 'bebes' 'papelaria'
 'tablets_impressao_imagem' 'brinquedos' 'telefonia_fixa'
 'ferramentas_jardim' 'fashion_bolsas_e_acessorios' 'eletroportateis'
 'consoles_games' 'audio' 'fashion_calcados' 'cool_stuff'
 'malas_acessorios' 'climatizacao' 'construcao_ferramentas_construcao'
 'moveis_cozinha_area_de_servico_jantar_e_jardim'
 'construcao_ferramentas_jardim' 'fashion_roupa_masculina' 'pet_shop'
 'moveis_escritorio' 'market_place' 'eletronicos' 'eletrodomesticos'
 'artigos_de_festas' 'casa_conforto' 'construcao_ferramentas_ferramentas'
 'agro_industria_e_comercio' 'moveis_colchao_e_estofado' 'livros_tecnicos'
 'casa_construcao' 'instrumentos_musicais' 'moveis_sala'
 'construcao_ferramentas_iluminacao' 'industria_comercio_e_negocios'
 'alimentos' 'artes' 'moveis_quarto' 'livros_interesse_ge

#### Identifying missing translations

In [42]:
translation_missing = set(product_cat_names) - set(product_cat_names_trans)
print(translation_missing)

{'portateis_cozinha_e_preparadores_de_alimentos', 'pc_gamer'}


In [43]:
df[df['product_category_name'].isin(translation_missing)]['product_category_name'].value_counts()

product_category_name
portateis_cozinha_e_preparadores_de_alimentos    14
pc_gamer                                          9
Name: count, dtype: int64

#### Adding missing translations to translation_df

In [44]:
translation_new_rows = pd.DataFrame({
    'product_category_name': [
        'pc_gamer',
        'portateis_cozinha_e_preparadores_de_alimentos'
    ],
    'product_category_name_english': [
        'gaming_pc',
        'portable_kitchen_and_food_preparators'
    ]
})

In [45]:
translation = pd.concat([translation, translation_new_rows], ignore_index=True)

In [46]:
product_cat_names_trans = translation['product_category_name'].unique()
print(len(product_cat_names_trans), product_cat_names_trans)

73 ['beleza_saude' 'informatica_acessorios' 'automotivo' 'cama_mesa_banho'
 'moveis_decoracao' 'esporte_lazer' 'perfumaria' 'utilidades_domesticas'
 'telefonia' 'relogios_presentes' 'alimentos_bebidas' 'bebes' 'papelaria'
 'tablets_impressao_imagem' 'brinquedos' 'telefonia_fixa'
 'ferramentas_jardim' 'fashion_bolsas_e_acessorios' 'eletroportateis'
 'consoles_games' 'audio' 'fashion_calcados' 'cool_stuff'
 'malas_acessorios' 'climatizacao' 'construcao_ferramentas_construcao'
 'moveis_cozinha_area_de_servico_jantar_e_jardim'
 'construcao_ferramentas_jardim' 'fashion_roupa_masculina' 'pet_shop'
 'moveis_escritorio' 'market_place' 'eletronicos' 'eletrodomesticos'
 'artigos_de_festas' 'casa_conforto' 'construcao_ferramentas_ferramentas'
 'agro_industria_e_comercio' 'moveis_colchao_e_estofado' 'livros_tecnicos'
 'casa_construcao' 'instrumentos_musicais' 'moveis_sala'
 'construcao_ferramentas_iluminacao' 'industria_comercio_e_negocios'
 'alimentos' 'artes' 'moveis_quarto' 'livros_interesse_ge

In [47]:
translation_mapping = dict(zip(
                            translation['product_category_name'],
                            translation['product_category_name_english']
                        ))
df['product_category_name'] = df['product_category_name'].map(translation_mapping)

In [48]:
print(df['product_category_name'].value_counts())

product_category_name
bed_bath_table                             11649
health_beauty                               9761
sports_leisure                              8731
furniture_decor                             8553
computers_accessories                       7897
housewares                                  7172
watches_gifts                               6063
telephony                                   4601
garden_tools                                4463
auto                                        4283
toys                                        4185
cool_stuff                                  3904
perfumery                                   3480
baby                                        3111
electronics                                 2807
stationery                                  2564
fashion_bags_accessories                    2106
pet_shop                                    2005
office_furniture                            1753
luggage_accessories                         114

In [49]:
print(df['product_category_name'].value_counts().sum())

113390


### Customer City

In [50]:
print(df['customer_city'].value_counts())
print(f"Total sum of vlaues in 'customer_city' is: {df['customer_city'].value_counts().sum()}")

customer_city
sao paulo                           17868
rio de janeiro                       7822
belo horizonte                       3134
brasilia                             2374
curitiba                             1768
campinas                             1686
porto alegre                         1589
salvador                             1435
guarulhos                            1348
sao bernardo do campo                1075
niteroi                              1016
santo andre                           888
osasco                                860
santos                                828
goiania                               812
sao jose dos campos                   780
fortaleza                             720
sorocaba                              705
recife                                669
florianopolis                         658
jundiai                               650
ribeirao preto                        589
contagem                              495
nova iguacu         

In [51]:
print(df['customer_city'].nunique())

4073


### Seller City

In [52]:
print(df['seller_city'].value_counts())
print(f"Total sum of vlaues in 'seller_city' is: {df['seller_city'].value_counts().sum()}")

seller_city
sao paulo                                   28297
ibitinga                                     8060
curitiba                                     3043
santo andre                                  3015
sao jose do rio preto                        2640
belo horizonte                               2527
rio de janeiro                               2373
ribeirao preto                               2271
maringa                                      2230
guarulhos                                    2011
piracicaba                                   1985
itaquaquecetuba                              1736
campinas                                     1393
salto                                        1374
praia grande                                 1318
campo limpo paulista                         1237
guariba                                      1178
sao bernardo do campo                        1152
jacarei                                       958
brasilia                              

In [53]:
print(df['seller_city'].nunique())

588


In [54]:
def clean_city(city):
    city = str(city).strip().lower()
    city = city.replace(' ', '_')
    city = city.replace('-', '_')
    city = city.replace('.', '')
    return city

In [55]:
df['customer_city_clean'] = df['customer_city'].apply(clean_city)

In [56]:
print(df['customer_city_clean'].nunique())

4072


In [57]:
customer_city_mapping = df.groupby('customer_city_clean')['customer_city'].unique()
duplicates = customer_city_mapping[customer_city_mapping.apply(lambda x: len(x) > 1)]
print(duplicates)

customer_city_clean
mogi_mirim    [mogi mirim, mogi-mirim]
Name: customer_city, dtype: object


In [58]:
df['customer_city'] = df['customer_city_clean']
df = df.drop(columns = ['customer_city_clean'])

In [59]:
print(df['customer_city'].nunique())

4072


In [60]:
df['seller_city_clean'] = df['seller_city'].apply(clean_city)

In [61]:
print(df['seller_city_clean'].nunique())

588


In [62]:
seller_city_mapping = df.groupby('seller_city_clean')['seller_city'].unique()
duplicates = seller_city_mapping[seller_city_mapping.apply(lambda x: len(x) > 1)]
print(duplicates)

Series([], Name: seller_city, dtype: object)


In [63]:
df['seller_city'] = df['seller_city_clean']
df = df.drop(columns = ['seller_city_clean'])

In [64]:
print(df['seller_city'].nunique())

588


In [65]:
# def correct_typos(df, column_name, threshold = 90):
#     unique_vals = df[column_name].unique()
#     corrections = {}
#     for val in unique_vals:
#         if corrections:
#             match_data = process.extractOne(val, corrections.keys(), scorer=fuzz.ratio)
#             if match_data is not None:
#                 match, score, _ = match_data
#                 if score >= threshold:
#                     corrections[val] = match
#                     continue
#         corrections[val] = val

#     cleaned_column = df[column_name].map(corrections)
#     return cleaned_column

# df['customer_city_clean'] = correct_typos(df, 'customer_city')
# print(df['customer_city_clean'].nunique())
# pd.reset_option('display.max_rows')
# changed_customer_cities = df[df['customer_city'] != df['customer_city_clean']]
# changed_customer_cities[['customer_city', 'customer_city_clean']].drop_duplicates()
# df = df.drop(columns = ['customer_city_clean'])

- The output of the above code cannot be confirmed wihout domain knowledge on Portugese/Brazillian Cities.
- Therefore, the assumption will be made that there are no typos and all values in 'customer_city' and 'seller_city' are valid
- Typos do not include lower/upper case or special characters. These have been dealth with already

## Date Features -> Identifying discrepancies/inconsistencies

In [66]:
for col in date_cols:
    print(f"{col} - Min date: {min(df[col])} | Max date: {max(df[col])}")

shipping_limit_date - Min date: 2016-10-08 10:34:01 | Max date: 2020-04-09 22:35:08
order_delivered_carrier_date - Min date: 2016-10-08 10:34:01 | Max date: 2018-09-11 19:48:28
order_delivered_customer_date - Min date: 2016-10-11 13:46:32 | Max date: 2018-10-17 13:22:46
order_estimated_delivery_date - Min date: 2016-10-27 00:00:00 | Max date: 2018-10-25 00:00:00
order_purchase_timestamp - Min date: 2016-10-03 09:44:50 | Max date: 2018-08-29 15:00:37
order_approved_at - Min date: 2016-10-04 09:43:32 | Max date: 2018-08-29 15:10:26


In [67]:
df[['shipping_limit_date', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'order_purchase_timestamp', 'order_approved_at']].isna().sum()

shipping_limit_date              0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
order_purchase_timestamp         0
order_approved_at                0
dtype: int64

In [68]:
df[['shipping_limit_date', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'order_purchase_timestamp', 'order_approved_at']].head()

,shipping_limit_date,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_timestamp,order_approved_at
0,2017-09-19 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,2017-09-13 08:59:02,2017-09-13 09:45:35
1,2017-07-05 02:44:11,2017-07-05 12:00:33,2017-07-13 20:39:29,2017-07-26,2017-06-28 11:52:20,2017-06-29 02:44:11
2,2018-05-23 10:56:25,2018-05-23 14:05:00,2018-06-04 18:34:26,2018-06-07,2018-05-18 10:25:53,2018-05-18 12:31:43
3,2017-08-07 18:55:08,2017-08-02 19:07:36,2017-08-09 21:26:33,2017-08-25,2017-08-01 18:38:42,2017-08-01 18:55:08
4,2017-08-16 22:05:11,2017-08-11 19:43:07,2017-08-24 20:04:21,2017-09-01,2017-08-10 21:48:40,2017-08-10 22:05:11


- `order_purchase_timestamp:` Date and Time the customer placed an order
- `order_approved_at:` Date and Time the order was approved by the seller
- `shipping_limit_date:` Deadline that the order must be shipped by
- `order_delivered_carrier_date:` Date and Time the order was delivered/picked up by the carrier
- `order_delivered_customer_date:` Date and time the customer actually received the order
- `order_estimated_delivery_date:` The estimated date that the order would have been delivered to the customer 

### Ensuring all orders were approved after they were purchased

In [69]:
approved_before_purchase = df[df['order_purchase_timestamp'] > df['order_approved_at']]
if approved_before_purchase.empty:
    print("There are no orders that were approved before purchasing. I.e all orders were approved after they were placed/purchased")
else:
    print(f"There are {len(approved_before_purchase)} orders that were approved before it/they were purhcased:\n {approved_before_purchase[['order_id', 'order_purchase_timestamp', 'order_approved_at']].head()}")

There are no orders that were approved before purchasing. I.e all orders were approved after they were placed/purchased


### Ensuring all orders were approved before they were delivered to the carrier

In [70]:
deliverd_to_carrier_before_approved = df[df['order_approved_at'] > df['order_delivered_carrier_date']]
if deliverd_to_carrier_before_approved.empty:
    print("There were no orders that were delivered to the carrier before it/they were approved. I.e All order were approved before being delivered to the carrier.")
else:
    print(f"There are {len(deliverd_to_carrier_before_approved)} orders that were delivered to a carrier before being approved:\n {deliverd_to_carrier_before_approved[['order_id', 'order_approved_at', 'order_delivered_carrier_date']].head()}")

There are 1582 orders that were delivered to a carrier before being approved:
                              order_id   order_approved_at  \
231  55471a53555cb61d9cd2ff895a278033 2018-07-05 16:09:39   
645  000576fe39319847cbb9d288c5617fa6 2018-07-05 16:35:48   
694  12f0d35d150a1645900980264e766cc6 2018-07-05 16:32:47   
699  b367cc1437c9d0b2794f759930e61eba 2018-04-24 18:42:20   
833  4cdbb26aa0dc8e7179f33edf59722fa8 2018-07-05 16:32:16   

    order_delivered_carrier_date  
231          2018-07-05 14:33:00  
645          2018-07-05 12:15:00  
694          2018-07-04 14:59:00  
699          2018-04-23 22:28:54  
833          2018-07-04 14:52:00  


- There are orders recorded for being delivered to the carrier before being approved
- This is impossible
- Due to the nature of the data (Third-Party Data), confirmation cannot be made whether these were typos or misinputs.
- These rows will therefore be dropped

In [71]:
df = df[df['order_approved_at'] <= df['order_delivered_carrier_date']]
df = df.reset_index(drop=True)
print(len(df))

111808


In [72]:
#confirming that deliverd_to_carrier_before_approved rows were dropped:
print((df['order_approved_at'] > df['order_delivered_carrier_date']).sum())

0


### Ensuring all orders were delivered to the carrier before they were delivered to the customer

In [73]:
delivered_to_customer_before_carrier = df[df['order_delivered_carrier_date'] > df['order_delivered_customer_date']]
if delivered_to_customer_before_carrier.empty:
    print("All orders were delivered to the carrier before the customer")
else:
    print(f"There were {len(delivered_to_customer_before_carrier)} orders delivered to the carrier before the customer:\n{delivered_to_customer_before_carrier[['order_id', 'order_delivered_carrier_date', 'order_delivered_customer_date']].head()}")

There were 58 orders delivered to the carrier before the customer:
                              order_id order_delivered_carrier_date  \
7737  29941903985f944b0ffc49c479c1547d          2017-06-09 15:07:29   
7738  29941903985f944b0ffc49c479c1547d          2017-06-09 15:07:29   
9762  a1abeb653a4d4cd1e142ccb8c82cd069          2017-07-28 16:57:58   
9763  a1abeb653a4d4cd1e142ccb8c82cd069          2017-07-28 16:57:58   
9764  a1abeb653a4d4cd1e142ccb8c82cd069          2017-07-28 16:57:58   

     order_delivered_customer_date  
7737           2017-06-02 11:09:16  
7738           2017-06-02 11:09:16  
9762           2017-07-25 19:32:56  
9763           2017-07-25 19:32:56  
9764           2017-07-25 19:32:56  


- There are orders recorded for being delivered to the customer before being delivered to the carrier
- This is impossible
- Due to the nature of the data (Third-Party Data), confirmation cannot be made whether these were typos or misinputs.
- These rows will therefore be dropped

In [74]:
df = df[df['order_delivered_carrier_date'] <= df['order_delivered_customer_date']]
df = df.reset_index(drop=True)
print(len(df))

111750


In [75]:
#confirming that deliverd_to_customer_before_carrier rows were dropped:
print((df['order_delivered_carrier_date'] > df['order_delivered_customer_date']).sum())

0


### Ensuring all orders were approved before they were delivered to the customer

In [76]:
delivered_to_customer_before_approved = df[df['order_approved_at'] > df['order_delivered_customer_date']]
if delivered_to_customer_before_approved.empty:
    print("There were no orders that were delivered to the customer before it/they were approved. I.e All order were approved before being delivered to the customer.")
else:
    print(f"There are {len(delivered_to_customer_before_approved)} orders that were delivered to a customer before being approved:\n {delivered_to_customer_before_approved[['order_id', 'order_approved_at', 'order_delivered_customer_date']].head()}")

There were no orders that were delivered to the customer before it/they were approved. I.e All order were approved before being delivered to the customer.


## Feature Engineering

### Approval Time

In [77]:
df['approval_time'] = df['order_approved_at'] - df['order_purchase_timestamp']
print(df[['order_purchase_timestamp', 'order_approved_at', 'approval_time']].head())

  order_purchase_timestamp   order_approved_at   approval_time
0      2017-09-13 08:59:02 2017-09-13 09:45:35 0 days 00:46:33
1      2017-06-28 11:52:20 2017-06-29 02:44:11 0 days 14:51:51
2      2018-05-18 10:25:53 2018-05-18 12:31:43 0 days 02:05:50
3      2017-08-01 18:38:42 2017-08-01 18:55:08 0 days 00:16:26
4      2017-08-10 21:48:40 2017-08-10 22:05:11 0 days 00:16:31


In [78]:
df['apporval_time_mins'] = df['approval_time'].dt.total_seconds()/60
print(df[['order_purchase_timestamp', 'order_approved_at', 'approval_time', 'apporval_time_mins']].head())

  order_purchase_timestamp   order_approved_at   approval_time  \
0      2017-09-13 08:59:02 2017-09-13 09:45:35 0 days 00:46:33   
1      2017-06-28 11:52:20 2017-06-29 02:44:11 0 days 14:51:51   
2      2018-05-18 10:25:53 2018-05-18 12:31:43 0 days 02:05:50   
3      2017-08-01 18:38:42 2017-08-01 18:55:08 0 days 00:16:26   
4      2017-08-10 21:48:40 2017-08-10 22:05:11 0 days 00:16:31   

   apporval_time_mins  
0           46.550000  
1          891.850000  
2          125.833333  
3           16.433333  
4           16.516667  


### Carrier Pickup Time

In [79]:
df['carrier_pickup_time'] = df['order_delivered_carrier_date'] - df['order_approved_at']
print(df[['order_approved_at', 'order_delivered_carrier_date', 'carrier_pickup_time']].head())

    order_approved_at order_delivered_carrier_date carrier_pickup_time
0 2017-09-13 09:45:35          2017-09-19 18:34:16     6 days 08:48:41
1 2017-06-29 02:44:11          2017-07-05 12:00:33     6 days 09:16:22
2 2018-05-18 12:31:43          2018-05-23 14:05:00     5 days 01:33:17
3 2017-08-01 18:55:08          2017-08-02 19:07:36     1 days 00:12:28
4 2017-08-10 22:05:11          2017-08-11 19:43:07     0 days 21:37:56


In [80]:
df['carrier_pickup_time_mins'] = df['carrier_pickup_time'].dt.total_seconds()/60
print(df[['order_approved_at', 'order_delivered_carrier_date', 'carrier_pickup_time', 'carrier_pickup_time_mins']].head())

    order_approved_at order_delivered_carrier_date carrier_pickup_time  \
0 2017-09-13 09:45:35          2017-09-19 18:34:16     6 days 08:48:41   
1 2017-06-29 02:44:11          2017-07-05 12:00:33     6 days 09:16:22   
2 2018-05-18 12:31:43          2018-05-23 14:05:00     5 days 01:33:17   
3 2017-08-01 18:55:08          2017-08-02 19:07:36     1 days 00:12:28   
4 2017-08-10 22:05:11          2017-08-11 19:43:07     0 days 21:37:56   

   carrier_pickup_time_mins  
0               9168.683333  
1               9196.366667  
2               7293.283333  
3               1452.466667  
4               1297.933333  


### From Carrier to Customer Time (Shipping Time)

In [81]:
df['shipping_time'] = df['order_delivered_customer_date'] - df['order_delivered_carrier_date']
print(df[['order_delivered_carrier_date', 'order_delivered_customer_date', 'shipping_time']].head())

  order_delivered_carrier_date order_delivered_customer_date    shipping_time
0          2017-09-19 18:34:16           2017-09-20 23:43:48  1 days 05:09:32
1          2017-07-05 12:00:33           2017-07-13 20:39:29  8 days 08:38:56
2          2018-05-23 14:05:00           2018-06-04 18:34:26 12 days 04:29:26
3          2017-08-02 19:07:36           2017-08-09 21:26:33  7 days 02:18:57
4          2017-08-11 19:43:07           2017-08-24 20:04:21 13 days 00:21:14


In [82]:
df['shipping_time_mins'] = df['shipping_time'].dt.total_seconds()/60
print(df[['order_delivered_carrier_date', 'order_delivered_customer_date', 'shipping_time', 'shipping_time_mins']].head())

  order_delivered_carrier_date order_delivered_customer_date    shipping_time  \
0          2017-09-19 18:34:16           2017-09-20 23:43:48  1 days 05:09:32   
1          2017-07-05 12:00:33           2017-07-13 20:39:29  8 days 08:38:56   
2          2018-05-23 14:05:00           2018-06-04 18:34:26 12 days 04:29:26   
3          2017-08-02 19:07:36           2017-08-09 21:26:33  7 days 02:18:57   
4          2017-08-11 19:43:07           2017-08-24 20:04:21 13 days 00:21:14   

   shipping_time_mins  
0         1749.533333  
1        12038.933333  
2        17549.433333  
3        10218.950000  
4        18741.233333  


### Total Delivery Time from purhcase

In [83]:
df['total_delivery_time'] = df['order_delivered_customer_date'] - df['order_purchase_timestamp']
print(df[['order_purchase_timestamp', 'order_delivered_customer_date', 'total_delivery_time']].head())

  order_purchase_timestamp order_delivered_customer_date total_delivery_time
0      2017-09-13 08:59:02           2017-09-20 23:43:48     7 days 14:44:46
1      2017-06-28 11:52:20           2017-07-13 20:39:29    15 days 08:47:09
2      2018-05-18 10:25:53           2018-06-04 18:34:26    17 days 08:08:33
3      2017-08-01 18:38:42           2017-08-09 21:26:33     8 days 02:47:51
4      2017-08-10 21:48:40           2017-08-24 20:04:21    13 days 22:15:41


In [84]:
df['total_delivery_time_mins'] = df['total_delivery_time'].dt.total_seconds()/60
print(df[['order_purchase_timestamp', 'order_delivered_customer_date', 'total_delivery_time', 'total_delivery_time_mins']].head())

  order_purchase_timestamp order_delivered_customer_date total_delivery_time  \
0      2017-09-13 08:59:02           2017-09-20 23:43:48     7 days 14:44:46   
1      2017-06-28 11:52:20           2017-07-13 20:39:29    15 days 08:47:09   
2      2018-05-18 10:25:53           2018-06-04 18:34:26    17 days 08:08:33   
3      2017-08-01 18:38:42           2017-08-09 21:26:33     8 days 02:47:51   
4      2017-08-10 21:48:40           2017-08-24 20:04:21    13 days 22:15:41   

   total_delivery_time_mins  
0              10964.766667  
1              22127.150000  
2              24968.550000  
3              11687.850000  
4              20055.683333  


### Delivery Delay Time

In [85]:
df['order_delay_time'] = df['order_delivered_customer_date'] - df['order_estimated_delivery_date']
df['order_delay_days'] = df['order_delay_time'].dt.total_seconds()/(3600 * 24)
df['order_delay_days'] = df['order_delay_days'].apply(lambda x: x if x > 0 else 0)
df['order_delay_time_mins'] = df['order_delay_time'].dt.total_seconds()/60
df['order_delay_time_mins'] = df['order_delay_time_mins'].apply(lambda x: x if x > 0 else 0)

print(f"There are {(df['order_delay_time_mins'] > 0).sum()} orders that were delivered late to the customer")
print(df[df['order_delay_time_mins'] > 0][['order_estimated_delivery_date', 'order_delivered_customer_date', 'order_delay_time', 'order_delay_days', 'order_delay_time_mins']].head())

There are 8841 orders that were delivered late to the customer
   order_estimated_delivery_date order_delivered_customer_date  \
5                     2017-08-24           2017-08-31 20:19:52   
47                    2017-11-03           2017-11-14 19:19:43   
54                    2017-08-23           2017-11-09 18:08:54   
60                    2017-11-03           2017-11-07 22:31:47   
71                    2018-04-04           2018-04-11 19:57:39   

   order_delay_time  order_delay_days  order_delay_time_mins  
5   7 days 20:19:52          7.847130           11299.866667  
47 11 days 19:19:43         11.805359           16999.716667  
54 78 days 18:08:54         78.756181          113408.900000  
60  4 days 22:31:47          4.938738            7111.783333  
71  7 days 19:57:39          7.831701           11277.650000  


In [86]:
df = df.drop(columns = {'approval_time', 'carrier_pickup_time', 'shipping_time', 'total_delivery_time', 'order_delay_time'})

## Identifying possible columns to turn to boolean dtype

In [87]:
possible_bool_cols = []
for col in df.columns:
    if df[col].nunique() == 2:
        possible_bool_cols.append(col)

if possible_bool_cols:
    print(f"possible boolean columns in the dataset:\n{possible_bool_cols}")
else:
    print("there are no boolean columns in the dataset.")

possible boolean columns in the dataset:
['order_status']


- Logically, a delivery can be ongoing, meaning that it is on the way to the customer
- It isnt depicted in the dataset as all orders and finalized and none ongoing, but the concept rings true
- Therefore, `order_status` will not be converted to a boolean data type just yet

# 3. Saving cleaned dataset to a csv file

In [88]:
df_clean = df

In [89]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111750 entries, 0 to 111749
Data columns (total 39 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       111750 non-null  object        
 1   order_item_id                  111750 non-null  int64         
 2   customer_id                    111750 non-null  object        
 3   customer_unique_id             111750 non-null  object        
 4   customer_zip_code_prefix       111750 non-null  int64         
 5   customer_city                  111750 non-null  object        
 6   customer_state                 111750 non-null  object        
 7   product_id                     111750 non-null  object        
 8   product_category_name          111750 non-null  object        
 9   product_name_length            111750 non-null  float64       
 10  product_description_length     111750 non-null  float64       
 11  

In [90]:
df_clean.to_csv("../Data/Cleaned_Data/Olist_Brazillian_Ecommerce_Dataset_Cleaned.csv", index = False, encoding = "utf-8")